<a href="https://colab.research.google.com/github/xanasa14/MLImplementations/blob/master/NaiveBayesFromOfflineDatasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import numpy as np, pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.metrics import confusion_matrix, accuracy_score
sns.set() # use seaborn plotting style

In [11]:
#importing our cancer dataset
dataset = pd.read_csv('/content/Errors.csv')


#X = dataset['radius mean', 'texture mean', 'perimeter mean', 'area_mean','smoothness_mean','compactness mean','concavity mean','concave points mean','symmetry mean','fractal dimension mean','radius se' ]
col_list = ['id', 'label', 'Description']
df = pd.read_csv("/content/Errors.csv", usecols=col_list)#dataset['diagnosis']
#Setting our X and Y for trainig and testing 
X  = df
Y = dataset['label']

In [15]:
#X and y split to train and test

#y = df['label']
#X = df.drop('label', axis=1)

In [19]:
print(X)
print(y)

    id                            Description
0    1              It froze for a long time 
1    2                A blue screen showed up
2    3                   Cant move the cursor
3    4                      Cant do anything 
4    5                        It says to wait
5    6                        Told me to wait
6    7                   It is not responding
7    8         The application cant be closed
8    9               Black Screen is showing 
9   10  Its been frozen for a very long time 
10  11                 A blue screen appeared
11  12              The cursor is not moving 
12  13        It does not let me do anything 
13  14  The operator said that I have to wait
14  15               A window of please wait 
15  16                 Nothing is responding 
16  17        The application is not closing 
0                    Reset the PC
1                    Reset the PC
2                    Reset the PC
3                    Reset the PC
4     Just wait until it responds
5 

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2)
print("\nX_train:\n")
print(X_train.head())
print(X_train.shape)

print("\nX_test:\n")
print(X_test.head())
print(X_test.shape)


X_train:

    id                            Description
0    1              It froze for a long time 
15  16                 Nothing is responding 
4    5                        It says to wait
10  11                 A blue screen appeared
9   10  Its been frozen for a very long time 
(13, 2)

X_test:

    id                      Description
2    3             Cant move the cursor
12  13  It does not let me do anything 
8    9         Black Screen is showing 
11  12        The cursor is not moving 
(4, 2)
